# X-ray Absorption Spectroscopy (XAS) Analysis at EuXFEL

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

![](misc/illustration.png)

## Plot the result with one line of code

In [ ]:
from xas_analysis import XasTim
%matplotlib inline

fig, ax = XasTim("/gpfs/exfel/exp/SCS/201831/p900048/raw/r0413").process(0, 50).\
    select('XGM', 0.1).select(['MCP1', 'MCP2', 'MCP3'], 1).plot_spectrum(n_bins=40)

## Data analysis in more details

```py
class XasDigitizer(XasProcessor):
     def __init__(self, run_folder, channels=('D', 'B', 'C', 'A'), 
                  pulse_separation=880e-9, interleaved_mode=False):
         """Initialization.
         
         :param str run_folder: full path of the run folder.
         :param tuple channels: names of AdqDigitizer channels which connects to MCP1 to MCP4.
         :param float pulse_separation: pulse separation in a train, in s.
         :param bool interleaved_mode: the resolution is improved by a factor of two 
             in the interleaved mode. Default = False.
         """
```

In [ ]:
# instantiate an `XasTim` object
xas = XasTim("/gpfs/exfel/exp/SCS/201831/p900048/raw/r0413")
xas.info()

## Correlation between I0 and I1

```py
def process(self, pulse_id0, n_pulses, *, use_apd=True, config=None):
    """Process the run data.

    :param int pulse_id0: first pulse ID.
    :param int n_pulses: number of pulses in a train.
    :param bool use_apd: use the integration calculated from the hardware.
    :param int peak_start: start position of the first peak. Ignored if use_apd == True.
    :param int peak_width: width of a peak. Ignored if use_apd == True.
    :param int background_end: end position of the background for the first peak. 
        Ignored if use_apd == True.
    :param int background_width: width of a background. Ignored if use_apd == True.
    """
```

In [ ]:
import time

t0 = time.time()

xas.process(0, 50)  # use 'APD' is way more faster than integrating over the raw data
# xas.process(0, 50, use_apd=False, 
#             peak_start=10285, peak_width=20, background_end=10280, background_width=280)

print("Processing time: {:.1f} seconds".format(time.time() - t0))

```py
@property
def data(self):
    """Get the pulse-resolved data in pandas.DataFrame.
   
    The data is not filtered! The signs of signals in MCP channels are flipped.
    """
```

In [ ]:
xas.data.head(10)

```py
def compute_total_absorption(self):
    """Compute absorption for all data.

    :return: total absorption data in pandas.DataFrame with index being the 
        MCP channel name and columns being:
        - muA: absorption mean;
        - sigmaA: absorption standard deviation;
        - muI0: I0 mean;
        - sigmaI0: I0 standard deviation;
        - weight: sum of I0 values;
        - muI1: I1 mean;
        - sigmaI1: I1 standard deviation;
        - corr: correlation coefficient between I0 and I1;
        - count: number of data.
    """
```

In [ ]:
# get absorption data as pandas.DataFrame
total_absorption = xas.compute_total_absorption()

total_absorption.head()

```py
def plot_correlation(self, channel=None, *, figsize=(8, 6), 
                     ms=6, alpha=0.05, n_bins=20):
    """Generate correlation plots.

    :param str channel: MCP channel name, e.g. MCP1, for visualizing 
        a single channel with four plots, or None (default) for  
        visualizing all the channels with one plot each.  
        Case insensitive. 
    :param tuple figsize: figure size.
    :param int ms: marker size for the scatter plots.
    :param float alpha: transparency for the scatter plots.
    :param int n_bins: number of bins for the histogram plots.
    """
```

In [ ]:
fig, ax = xas.plot_correlation()

In [ ]:
fig, ax = xas.plot_correlation('mcp1')

In [ ]:
fig, ax = xas.plot_correlation('mcp2')

In [ ]:
fig, ax = xas.plot_correlation('mcp3')

In [ ]:
fig, ax = xas.plot_correlation('mcp4')

## Absorption spectrum

```py
def compute_spectrum(self, n_bins=20):
    """Calculate spectrum.

    :param int n_bins: number of energy bins.

    :return: spectrum data in pandas.DataFrame with index being the 
        energy bin range and columns being;
        - energy: central energy of each bin;
        - count: number of data points for each energy bin;
        - muXGM, muMCP1, muMCP2, muMCP3: intensity mean;
        - sigmaXGM, sigmaMCP1, sigmaMCP2, sigmaMCP3: standard deviation mean;
        - muA1, muA2, muA3: absorption mean;
        - sigmaA1, sigmaA2, sigmaA3: absorption standard deviation;
        - corrMCP1, corrMCP2, corrMCP3: correlation between XGM and MCP.
    """
```

In [ ]:
spectrum = xas.compute_spectrum(n_bins=30)
spectrum

```py
def plot_spectrum(self, channel=None, *, figsize=(6, 4.5), capsize=4, 
                  n_bins=20, use_transmission=False):
    """Generate spectrum plots.

    :param str channel: MCP channel name, e.g. MCP1, for visualizing
        a single channel, or None (default) for visualizing MCP1-3 
        altogether. Case insensitive.
    :param tuple figsize: figure size.
    :param int capsize: cap size for the error bar.
    :param int n_bins: number of energy bins.
    :param bool use_transmission: False for plotting energy vs. 
        absorption, while True for plotting energy vs. I1. 
        Default = False
    """
```

In [ ]:
fig, ax = xas.plot_spectrum(n_bins=40)

In [ ]:
fig, ax = xas.plot_spectrum(use_transmission=True, n_bins=40)

In [ ]:
fig, ax = xas.plot_spectrum('mcp2', n_bins=40)

In [ ]:
fig, ax = xas.plot_spectrum('mcp2', use_transmission=True, n_bins=40)

## Sanity check

```py
def plot_xgm_run(self, *, figsize=(8, 5.6)):
    """Plot the train resolved data from XGM.
    
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, (ax11, ax12, ax2) = xas.plot_xgm_run()

```py
def plot_xgm_train(self, *, index=0, train_id=None, figsize=(8, 5.6)):
    """Plot xgm measurement in a given train.

    :param int index: train index. Ignored if train_id is given.
    :param int train_id: train ID.
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, ax = xas.plot_xgm_train(index=10)

```py
def plot_digitizer_train(self, *, index=0, train_id=None, figsize=(8, 11.2),
                         x_min=None, x_max=None):
    """Plot digitizer signals in a given train.

    :param int index: train index. Ignored if train_id is given.
    :param int train_id: train ID.
    :param tuple figsize: figure size.
    :param int x_min: minimum sample ID.
    :param int x_max: maximum sample ID.
    """
```

In [ ]:
fig, axes = xas.plot_digitizer_train(index=200, x_min=0, x_max=120000)